In [ ]:
import docx
import os, re
from collections import Counter

# Data loading code

Counter(labels)


Counter({0: 168, 1: 195})

In [ ]:
#!pip3 install tensorflow-gpu
!pip3 install --quiet tensorflow-hub
!pip3 install tensorflow_text #>=2.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 68.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing instal

In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")


Compute embeddings in batches of 50 texts:

In [ ]:

all_embedded_texts = []
n=50


chunks=[texts[i:i + n] for i in range(0, len(texts), n)]
for chunk in chunks:
   all_embedded_texts.extend( embed(chunk).numpy() )



In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
import numpy as np
from sklearn.svm import NuSVC
from collections import Counter
import pickle

kernel_type = 'rbf'

runtime_name='uselarge3'

labels = np.array(labels)
embeddings = np.array(all_embedded_texts)

true_labels = [] # ground truth
predictions = [] # model predictions

kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(texts):

  X_train, X_test = embeddings[train_index], embeddings[test_index]
  y_train, y_test = labels[train_index], labels[test_index]

  print(Counter(y_train))

  clf = NuSVC(nu=0.5, kernel=kernel_type)

  model = clf.fit(X_train, y_train.ravel())
  y_pred = model.predict(X_test).tolist()

  predictions.append( y_pred )
  true_labels.append( y_test )


print( classification_report(np.concatenate(true_labels).ravel().tolist(), np.concatenate(predictions).ravel().tolist() ) )


Counter({1: 177, 0: 149})
Counter({1: 177, 0: 149})
Counter({1: 171, 0: 155})
Counter({1: 176, 0: 151})
Counter({1: 176, 0: 151})
Counter({1: 180, 0: 147})
Counter({1: 174, 0: 153})
Counter({1: 177, 0: 150})
Counter({1: 174, 0: 153})
Counter({1: 173, 0: 154})
              precision    recall  f1-score   support

           0       0.76      0.82      0.79       168
           1       0.83      0.77      0.80       195

    accuracy                           0.79       363
   macro avg       0.79      0.79      0.79       363
weighted avg       0.80      0.79      0.79       363



Print the number of parameters of the SVM classifier:

In [ ]:
def n_params(model):
    """Return total number of parameters in a
    Scikit-Learn model.

    This works for the following model types:
     - sklearn.neural_network.MLPClassifier
     - sklearn.neural_network.MLPRegressor
     - sklearn.linear_model.LinearRegression
     - and maybe some others
    """
    return (
            sum([a.size for a in model.dual_coef_]) +
            model.support_vectors_.shape[0] * model.support_vectors_.shape[1]
            )

n_params(model)

129276

In [ ]:
import tensorflow as tf

trainable_params = sum(count_params(layer) for layer in embed.trainable_weights)
non_trainable_params = sum(count_params(layer) for layer in embed.non_trainable_weights)


print the number of parameters of the USE encoder, convert it to Keras model:

In [ ]:
import tensorflow as tf

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"

sent1_input = tf.keras.layers.Input(shape=[], name='sent1',dtype=tf.string)
use_model = hub.KerasLayer(module_url, trainable= False)
output = use_model(sent1_input)

In [ ]:
model = tf.keras.Model(inputs=[sent1_input], outputs=output)
model.compile(loss='categorical_crossentropy',
	optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sent1 (InputLayer)          [(None,)]                 0         
                                                                 
 keras_layer (KerasLayer)    (None, 512)               85213184  
                                                                 
Total params: 85213184 (325.06 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 85213184 (325.06 MB)
_________________________________________________________________
